In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [2]:
spark = (
    SparkSession.builder.appName("Read S3 File")
    .master("spark://localhost:7077")
    .config(
        "spark.jars",
        "/opt/bitnami/spark/jars/hadoop-aws-3.3.4.jar,/opt/bitnami/spark/jars/aws-java-sdk-bundle-1.12.262.jar",
    )
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/12/04 07:43:26 WARN Utils: Your hostname, cpPC resolves to a loopback address: 127.0.1.1; using 172.25.249.11 instead (on interface eth0)
23/12/04 07:43:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/12/04 07:43:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark.conf.set("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com")

In [4]:
df = spark.range(1000).toDF("number")
df.show(2)

+------+
|number|
+------+
|     0|
|     1|
+------+
only showing top 2 rows



In [5]:
# Read
taxi_zone_df = (
    spark.read.option("inferSchema", "true")
    .option("header", "true")
    .csv("s3a://de-666ne92p/tmp/taxi+_zone_lookup.csv")
    .cache()
)

23/12/04 07:43:44 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [6]:
taxi_zone_df.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [7]:
date_dim_df = (
    spark.read.option("inferSchema", "true")
    .option("header", "true")
    .csv("s3a://de-666ne92p/tmp/date_dim.csv")
    .cache()
)

In [8]:
date_dim_df.show(2)

+--------+----------+-----------+--------+------------+-----------+------------+-----+----------+-------+----+----------+----------+----------+
|date_key|      date|day_of_week|day_name|day_of_month|day_of_year|week_of_year|month|month_name|quarter|year|is_holiday|is_weekend|is_workday|
+--------+----------+-----------+--------+------------+-----------+------------+-----+----------+-------+----+----------+----------+----------+
|   10957|2000-01-01|          6|Saturday|          31|          1|          52|    1|   January|      1|2000|      true|      true|     false|
|   10958|2000-01-02|          0|  Sunday|          31|          2|          52|    1|   January|      1|2000|     false|      true|     false|
+--------+----------+-----------+--------+------------+-----------+------------+-----+----------+-------+----+----------+----------+----------+
only showing top 2 rows



In [9]:
def dimention_df():
    def taxi_zone_df():
        return (
            spark.read.option("inferSchema", "true")
            .option("header", "true")
            .csv("s3a://de-666ne92p/tmp/taxi+_zone_lookup.csv")
            .cache()
        )

    def date_dim_df():
        return (
            spark.read.option("inferSchema", "true")
            .option("header", "true")
            .csv("s3a://de-666ne92p/tmp/date_dim.csv")
            .cache()
        )

    def vendor_df():
        schema = StructType(
            [
                StructField("VENDOR_KEY", IntegerType(), True),
                StructField("VENDOR_ID", IntegerType(), True),
                StructField("VENDOR_NAME", StringType(), True),
            ]
        )

        # Your CSV-like data
        data = [(1, 1, "Creative Mobile Technologies, LLC"),
                (2, 2, "VeriFone Inc.")]

        # Create a DataFrame
        return spark.createDataFrame(data, schema=schema)

    def trip_type_df():
        schema = StructType(
            [
                StructField("TRIP_TYPE_KEY", IntegerType(), True),
                StructField("TRIP_TYPE_ID", IntegerType(), True),
                StructField("TRIP_TYPE_NAME", StringType(), True),
            ]
        )

        data = [(1, 1, "Street-hail"), (2, 2, "Dispatch")]

        return spark.createDataFrame(data, schema=schema)

    def rate_code_df():
        schema = StructType(
            [
                StructField("RATE_CODE_KEY", IntegerType(), True),
                StructField("RATE_CODE_ID", IntegerType(), True),
                StructField("RATE_CODE_EFFECT", StringType(), True),
            ]
        )

        data = [
            (1, 1, "Standard rate"),
            (2, 2, "JFK"),
            (3, 3, "Newark"),
            (4, 4, "Nassau or Westchester"),
            (5, 5, "Negotiated fare"),
            (6, 6, "Group ride"),
        ]

        return spark.createDataFrame(data, schema=schema)

    def payment_type_df():
        schema = StructType(
            [
                StructField("PAYMENT_TYPE_KEY", IntegerType(), True),
                StructField("PAYMENT_TYPE_ID", IntegerType(), True),
                StructField("PAYMENT_TYPE_NAME", StringType(), True),
            ]
        )

        data = [
            (1, 1, "Credit card"),
            (2, 2, "Cash"),
            (3, 3, "No charge"),
            (4, 4, "Dispute"),
            (5, 5, "Unknown"),
            (6, 6, "Voided trip"),
        ]

        return spark.createDataFrame(data, schema=schema)

    return (
        taxi_zone_df(),
        date_dim_df(),
        vendor_df(),
        trip_type_df(),
        rate_code_df(),
        payment_type_df(),
    )

In [10]:
(
    taxi_zone_df,
    date_dim_df,
    vendor_df,
    trip_type_df,
    rate_code_df,
    payment_type_df,
) = dimention_df()

taxi_zone_df.show(5)

23/12/04 07:44:06 WARN CacheManager: Asked to cache already cached data.
23/12/04 07:44:10 WARN CacheManager: Asked to cache already cached data.        


+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [11]:
date_dim_df.show(5)

+--------+----------+-----------+---------+------------+-----------+------------+-----+----------+-------+----+----------+----------+----------+
|date_key|      date|day_of_week| day_name|day_of_month|day_of_year|week_of_year|month|month_name|quarter|year|is_holiday|is_weekend|is_workday|
+--------+----------+-----------+---------+------------+-----------+------------+-----+----------+-------+----+----------+----------+----------+
|   10957|2000-01-01|          6| Saturday|          31|          1|          52|    1|   January|      1|2000|      true|      true|     false|
|   10958|2000-01-02|          0|   Sunday|          31|          2|          52|    1|   January|      1|2000|     false|      true|     false|
|   10959|2000-01-03|          1|   Monday|          31|          3|           1|    1|   January|      1|2000|     false|     false|      true|
|   10960|2000-01-04|          2|  Tuesday|          31|          4|           1|    1|   January|      1|2000|     false|     fal

In [12]:
vendor_df.show(5)

+----------+---------+--------------------+
|VENDOR_KEY|VENDOR_ID|         VENDOR_NAME|
+----------+---------+--------------------+
|         1|        1|Creative Mobile T...|
|         2|        2|       VeriFone Inc.|
+----------+---------+--------------------+



In [13]:
trip_type_df.show(5)

+-------------+------------+--------------+
|TRIP_TYPE_KEY|TRIP_TYPE_ID|TRIP_TYPE_NAME|
+-------------+------------+--------------+
|            1|           1|   Street-hail|
|            2|           2|      Dispatch|
+-------------+------------+--------------+



In [14]:
rate_code_df.show(5)

+-------------+------------+--------------------+
|RATE_CODE_KEY|RATE_CODE_ID|    RATE_CODE_EFFECT|
+-------------+------------+--------------------+
|            1|           1|       Standard rate|
|            2|           2|                 JFK|
|            3|           3|              Newark|
|            4|           4|Nassau or Westche...|
|            5|           5|     Negotiated fare|
+-------------+------------+--------------------+
only showing top 5 rows



In [15]:
payment_type_df.show(5)

+----------------+---------------+-----------------+
|PAYMENT_TYPE_KEY|PAYMENT_TYPE_ID|PAYMENT_TYPE_NAME|
+----------------+---------------+-----------------+
|               1|              1|      Credit card|
|               2|              2|             Cash|
|               3|              3|        No charge|
|               4|              4|          Dispute|
|               5|              5|          Unknown|
+----------------+---------------+-----------------+
only showing top 5 rows



In [20]:
green_trip_data_df = (
    spark.read.format("parquet")
    .load("s3a://de-666ne92p/tmp/green_tripdata_2023-01.parquet")
    .cache()
)

23/12/04 07:47:19 WARN CacheManager: Asked to cache already cached data.        


In [21]:
green_trip_data_df.show(5)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2023-01-01 00:26:10|  2023-01-01 00:37:11|                 N|       1.0|         166|         143|            1.0|         2.58|       14.9|  1.0|    0.

In [22]:
green_trip_data_df.count()

68211

In [36]:
# Uppercase colume name in green trip data
for col_name in green_trip_data_df.columns:
    green_trip_data_df = green_trip_data_df.withColumnRenamed(
        col_name, col_name.upper()
    )

green_trip_data_df.printSchema()

root
 |-- VENDORID: long (nullable = true)
 |-- LPEP_PICKUP_DATETIME: timestamp_ntz (nullable = true)
 |-- LPEP_DROPOFF_DATETIME: timestamp_ntz (nullable = true)
 |-- STORE_AND_FWD_FLAG: string (nullable = true)
 |-- RATECODEID: double (nullable = true)
 |-- PULOCATIONID: long (nullable = true)
 |-- DOLOCATIONID: long (nullable = true)
 |-- PASSENGER_COUNT: double (nullable = true)
 |-- TRIP_DISTANCE: double (nullable = true)
 |-- FARE_AMOUNT: double (nullable = true)
 |-- EXTRA: double (nullable = true)
 |-- MTA_TAX: double (nullable = true)
 |-- TIP_AMOUNT: double (nullable = true)
 |-- TOLLS_AMOUNT: double (nullable = true)
 |-- EHAIL_FEE: integer (nullable = true)
 |-- IMPROVEMENT_SURCHARGE: double (nullable = true)
 |-- TOTAL_AMOUNT: double (nullable = true)
 |-- PAYMENT_TYPE: double (nullable = true)
 |-- TRIP_TYPE: double (nullable = true)
 |-- CONGESTION_SURCHARGE: double (nullable = true)



In [41]:
from pyspark.sql.functions import col, to_date  # noqa: E402

fact_green_trip_data_df = (
    green_trip_data_df.join(
        vendor_df, green_trip_data_df["VENDORID"] == vendor_df["VENDOR_ID"], "inner"
    )
    .join(
        trip_type_df,
        green_trip_data_df["TRIP_TYPE"] == trip_type_df["TRIP_TYPE_ID"],
        "inner",
    )
    .join(
        rate_code_df,
        green_trip_data_df["RATECODEID"] == rate_code_df["RATE_CODE_ID"],
        "inner",
    )
    .join(
        payment_type_df,
        green_trip_data_df["PAYMENT_TYPE"] == payment_type_df["PAYMENT_TYPE_ID"],
        "inner",
    )
    .join(
        taxi_zone_df.alias("taxi_zone_df_pu"),
        green_trip_data_df["PULOCATIONID"] == col(
            "taxi_zone_df_pu.LOCATIONID"),
        "inner",
    )
    .join(
        taxi_zone_df.alias("taxi_zone_df_do"),
        green_trip_data_df["DOLOCATIONID"] == col(
            "taxi_zone_df_do.LOCATIONID"),
        "inner",
    ).join(
        date_dim_df.alias("date_dim_df_pu"),
        to_date(green_trip_data_df["LPEP_PICKUP_DATETIME"]) == col(
            "date_dim_df_pu.DATE"),
        "inner",
    ).join(
        date_dim_df.alias("date_dim_df_do"),
        to_date(green_trip_data_df["LPEP_DROPOFF_DATETIME"]) == col(
            "date_dim_df_do.DATE"),
        "inner",
    )
)
fact_green_trip_data_df.show(5)
fact_green_trip_data_df.count()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+----------+---------+-------------+-------------+------------+--------------+-------------+------------+----------------+----------------+---------------+-----------------+----------+---------+--------------------+------------+----------+---------+--------------------+------------+--------+----------+-----------+--------+------------+-----------+------------+-----+----------+-------+----+----------+----------+----------+--------+----------+-----------+--------+------------+-----------+------------+-----+----------+-------+----+----------+----------+----------+
|VENDORID|LPEP_PICKUP_DATETIME|LPEP_DROPOFF_DATETIME|STORE_AND_FWD_FLAG|RATECODEID|PULOCATIONID|DOLOCATIONID|PASSENGER_COUNT|TRIP_DISTANCE|FARE_AMOUNT|EXTR

63877

In [31]:
from pyspark.sql.functions import col  # noqa: E402

green_trip_data_df.where(col("TRIP_DISTANCE").isNull()).count()

0